In [1]:
import os
import numpy as np
import re

from gurobipy import Model, GRB, GurobiError
import matplotlib.pyplot as plt
import numpy as np


In [3]:
DIR = 'data'
#n = 17
problem = 'rbg323.atsp' # 'ftv64.atsp' #'br17.atsp'
file_name = os.path.join(DIR, problem)


numeric_part = re.search(r'\d+', problem).group()

# Convert the extracted numeric part to an integer
n = int(numeric_part)
print(n)

323


In [4]:
if n == 64:
    n = 65

In [5]:
with open(file_name, 'r') as file:
    content = file.read()

In [6]:
data_lines = content.split('\n')

start_index = data_lines.index('EDGE_WEIGHT_SECTION')

# Extract the data lines after 'EDGE_WEIGHT_SECTION'
edge_weight_lines = data_lines[start_index + 1:][:-2]

# Parse the distance matrix (lines are not rows)
distance_matrix = [list(map(int, line.split())) for line in edge_weight_lines if line.strip()]

def flatten(l):
    return [item for sublist in l for item in sublist]

distance_list = flatten(distance_matrix)
print(len(distance_list))

my_matrix = np.array(distance_list).reshape((n, n))
my_matrix.shape

104329


(323, 323)

In [7]:
V = range(n)

cost = {
    (i, j): my_matrix[i,j]
    for i in V for j in V if i != j
}

A = list(cost.keys())

In [8]:
def subtour_starting_at(vertex):
    tour = [vertex]
    current = next_after(vertex)

    while current != vertex:
        tour.append(current)
        current = next_after(current)

    return tour

def add_sec_for(subtour):
    #print('Added a violated SEC')
    global subtours_size
    
    n = len(set(subtour))
    #print(subtours_size)
    if n in subtours_size:
        subtours_size[n] = subtours_size[n] + 1
    else:
        subtours_size[n] = 1
    #print(set(subtour))
    m.cbLazy(sum(x[i, j] for i in subtour for j in set(V) - set(subtour)) >= 1)

def next_after(i):
    for j in V:
        if j == i:
            continue
        try:
            if m.cbGetSolution(x[i, j]) > 0.9:
                return j
        except GurobiError:
            if x[i, j].X > 0.9:
                return j
    
    assert False, f"Vertex {i} has no successor"

def callback(what, where):
    if where != GRB.Callback.MIPSOL:
        return

    remaining = set(V)

    while len(remaining) > 0:
        current = next(iter(remaining))
        subtour = subtour_starting_at(vertex=current)

        if len(subtour) == n:
            return
        
        add_sec_for(subtour)

        remaining -= set(subtour)

# Baseline model

In [ ]:
def my_model(cardinality = None):
    m = Model()
    x = m.addVars(A, vtype=GRB.BINARY, name='x', obj=cost)
    m.addConstrs((x.sum(i, '*') == 1 for i in V), name='out_degree')
    m.addConstrs((x.sum('*', i) == 1 for i in V), 'in_degree');
    m.optimize(callback)

    # Conditionally add cardinality constraints if the cardinality is set to 2
    if self.cardinality == 2 or self.cardinality == 3:
         m.addConstrs((x[i, j] + x[j, i] <= 1 for i,j in V if i != j), 'cardinality two')

    # Add constraints to ensure x(i,j) + x(j,k) +x(k,i) <= 2
    if self.cardinality == 3:
        m.addConstrs((x[i, j] + x[j, k] + x[k, i]<= 2 for i, j, k in V if i != j and j != k and k != i), 'cardinality three')


In [ ]:
import timeit

execution_times = timeit.repeat(stmt=my_model, setup="pass", repeat=5, number=1)

# Calculate mean and standard deviation
mean_time = sum(execution_times) / len(execution_times)
std_time = (sum((time - mean_time) ** 2 for time in execution_times) / len(execution_times)) ** 0.5

print(f"Mean Execution Time: {mean_time} seconds")
print(f"Standard Deviation: {std_time} seconds")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 646 rows, 104006 columns and 208012 nonzeros
Model fingerprint: 0x6de01d01
Variable types: 0 continuous, 104006 integer (104006 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5884.0000000
Presolve time: 0.13s
Presolved: 646 rows, 104006 columns, 208012 nonzeros
Variable types: 0 continuous, 104006 integer (104006 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      0s
       1        855   2.5000000e+08   1.2350000e+02      0s
       2       1423   2.3800001e+08   2.4050000e+02      0s
       3       1713   2.1800003e+08   

# Fractional method

In [ ]:
def callback(what, where):
    if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
        return
    
    if where == GRB.Callback.MIPNODE and m.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
        return
    
    set_capacity()

    source = G.vertex(0)
    added  = set()

    for i in range(1, n):
        if i in added:
            continue

        sink = G.vertex(i)
        res = boykov_kolmogorov_max_flow(g=G, source=source, target=sink, capacity=cap)

        # Create an edge property map quickly by
        # copying an existing one.
        flow = res.copy()

        # The value held by the property map is in
        # the .a member. Because capacity == flow + residuals
        # we must write:
        flow.a = cap.a - res.a

        maxflow = sum(flow[a] for a in sink.in_edges())

        if maxflow < 1 - 1e-6:
            print(f"Violated SEC. Flow = {maxflow:.3f} < 1")
            cut = min_st_cut(g=G, source=source, capacity=cap, residual=res)

            assert cut[source] == True
            assert cut[sink] == False

            subtour = [j for j in V if cut[G.vertex(j)] == False]

            assert len(subtour) < n

            add_sec_for(subtour)

            added = added.union(subtour)
            
def set_capacity():
    for e in G.edges():
        i, j = e.source(), e.target()

        try:
            xval = m.cbGetSolution(x[i,j])
        except GurobiError:
            xval = m.cbGetNodeRel(x[i,j])

        cap[e] = xval

In [ ]:
G = complete_graph(N=n, self_loops=False, directed=True)
cap = G.new_edge_property(value_type='double')
G.edge_properties['cap'] = cap

m = Model()
x = m.addVars(A, vtype=GRB.BINARY, obj=cost, name='x')
m.addConstrs(x.sum(i, '*') == 1 for i in V)
m.addConstrs(x.sum('*', i) == 1 for i in V)
m.setParam(GRB.Param.LazyConstraints, 1)

NameError: name 'complete_graph' is not defined

In [ ]:
m.optimize(callback=callback)

In [ ]:
import site

# List of directories where packages are installed
print(site.getsitepackages())

# Question 2
